This is the notebook, set to produice prediction for the model with the highest accuracy on the private leaderboard. 

In [1]:
#@title Installation
!pip install --quie ipdb # debug
!pip install --quie ipython-autotime  # timming
!pip install --quie optuna  # hyperparaeters

##  <font color='red'>You need to run the cell below twice before to proceed </font>
If you can an error, running the cell bellow, please just re-run it. 

In [18]:
# @title Imports
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import cvxopt
np.random.seed(54321)

import ipdb

import optuna

from sklearn.model_selection import KFold 
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.preprocessing import OneHotEncoder

from kernels.basic import Linear, Polynomial
from kernels.regular import rbf_kernel, Exponential, Laplacian, RationalQuadratic, \
                            InverseMultiquadratic, Cauchy, TStudent, ANOVA, Fourier, Tanimoto, Sorensen

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 6.07 ms


In [19]:
X_train=pd.read_csv('./data/Xtr.csv', sep=',') #we use this dataset to train our model
Y_train=pd.read_csv('./data/Ytr.csv', sep=',') #we use this dataset to train our model
X_test=pd.read_csv('./data/Xte.csv', sep=',') #we will use this data set later to validate our model

# X_train_mat=pd.read_csv('./data/Xtr_mat100.csv', sep=',') #we use this dataset to train our model
# X_test_mat=pd.read_csv('./data/Xte_mat100.csv', sep=',') #we will use this data set later to validate our model

time: 14.1 ms


In [20]:
X_train.tail()

,Id,seq
1995,1995,TAACTTTTGACAGGTCAGAATACAAAACTGATTTATTTACAGTGTC...
1996,1996,ACGCCCATTCCGCCCTGCTAAGCCTCGCCCATTACATCCAGACTGC...
1997,1997,TGGCTACTAGCTAGAGATAGCATCTCTCTGTGGACAACTCTCCAGC...
1998,1998,CCCAGCTGTCAAAAAGCAGCCCAAAGGAAGCTCACGGTGTGCCGGC...
1999,1999,TGCTAGTTGATGAAACAATAACTGCTAAAAGGTATACAGCCATGTC...


time: 9.69 ms


In [21]:
print('The shape of the X_train dataset is:',X_train.shape)
print('The shape of the Y_train dataset is:',Y_train.shape)

The shape of the X_train dataset is: (2000, 2)
The shape of the Y_train dataset is: (2000, 2)
time: 1.07 ms


# Models

In [22]:
# Ridge Regression (RR)

class solveRR():
    def __init__(self, X, y, lam=0.1):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
            
    def fit(self):
        
        X = self.X
        y = self.y
        lam = self.lam 
        
        n, p = X.shape
        assert (len(y) == n)

        A = (X.T.dot(X)) + np.eye(p)*lam*n
        b = X.T.dot(y)
        
        self.beta = np.linalg.solve(A, b)
        
        return self.beta
    
        
    def predict(self, X, threshold=.5):
        return np.where(X.dot(self.beta) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)
    

# Weighted Ridge Regression (WRR)
class solveWRR():
    def __init__(self, X, y, w, lam=0.1):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
        self.w = w
    
    def fit(self):
        
        X = self.X
        y = self.y
        lam = self.lam 
        w = self.w
        
        n, p = X.shape
        assert (len(y) == len(w) == n)

        y1 = np.sqrt(w) * y
        X1 = (np.sqrt(w) * X.T).T
        
        # Hint:
        # Find y1 and X1 such that:
        
        self.beta = solveRR(X1, y1, lam).fit()
                
        return self.beta
    
        
    def predict(self, X, threshold):
        return np.where(X.dot(self.beta) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)
    

# Logistic Ridge Regression (LRR)
class solveLRR():
    def __init__(self, X, y, lam=0.1):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
    
    def fit(self):
        
        X = self.X
        y = self.y
        
        n, p = X.shape
        assert (len(y) == n)
    
        lam = self.lam 
        max_iter = 50
        eps = 1e-3
        sigmoid = lambda a: 1/(1 + np.exp(-a))
        
        
        
        # Initialize
        self.beta = np.zeros(p)

        # Hint: Use IRLS
        for i in range(max_iter):
            beta_old = self.beta
            f = X.dot(beta_old)
            w = sigmoid(f) * sigmoid(-f)
            z = f + y / sigmoid(y*f)
            self.beta = solveWRR(X, z, w, 2*lam).fit()
            # Break condition (achieved convergence)
            #if np.sum((beta-beta_old)**2) < eps:
            #    break                
        return self.beta
    
        
    def predict(self, X, threshold):
        return np.where(X.dot(self.beta) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

time: 115 ms


# Kernel based Models

We're just applying the kernel trick to the above models, to have models that can fit non linear like data.

In [23]:
# Weighted Kernel Ridge Regression 

class ksolveRR_2():
    def __init__(self, X, y, lam= 0.1, sample_weights = None, kernel = None):
        self.alpha = None
        self.X = X
        self.y = y
        self.lam = lam
        self.kernel = kernel
        self.sample_weights = sample_weights
            
    
    def fit(self):
        if self.sample_weights is not None:
            self.X *= self.sample_weights[:, None]
        
        X = self.X
        y = self.y
        lam = self.lam
        
        n, p = X.shape
        assert (len(y) == n)
        
        A = self.kernel(X, X)+n*self.lam*np.eye(n)
        self.alpha = np.linalg.solve(A, y)
        
        return self
    
        
    def predict(self, X, threshold=.5):
        K_x = self.kernel(X, self.X)
        return np.where(K_x.dot(self.alpha) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

time: 17.2 ms


In [24]:
# Kernel Ridge Regression
class ksolveRR():
    def __init__(self, X, y, lam= 0.0001, kernel=None):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
        self.kernel = kernel
            
    
    def fit(self):
        X = self.X
        y = self.y
        lam = self.lam 
        
        n, p = X.shape
        assert (len(y) == n)
        
#         if self.sigma is None:
#             self.sigma = sigma_from_median(X)
            
#         A = self.kernel(X, X, self.sigma)+n*self.lam*np.eye(n)
        A = self.kernel(X, X) + n*self.lam*np.eye(n)
        self.alpha = np.linalg.solve(A, y)
        
        return self.beta
    
        
    def predict(self, X, threshold=.5):
#         K_x = self.kernel(X, self.X, self.sigma)
        K_x = self.kernel(X, self.X)
        return np.where(K_x.dot(self.alpha) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

time: 15.2 ms


In [25]:
# Logistic Ridge Regression (LRR)
class ksolveLRR():
    def __init__(self, X, y, lam = 0.1, sigma = 4, max_iter=100, tol=1e-5, kernel=None):
        self.alpha = None
        self.X = X
        self.y = y
        self.lam = lam
        
        self.kernel = kernel
        
        self.sigma = sigma
        self.max_iter = max_iter
        self.tol = tol
        
    
    def fit(self):
        
        X = self.X
        y = self.y
        
        n, p = X.shape
        assert (len(y) == n)

        sigmoid = lambda a: 1/(1 + np.exp(-a))
        
        K = self.kernel(X, X)

        # Initialize
        alpha = np.zeros(n)
        
        # Hint: Use IRLS
        for n_iter in range(self.max_iter):
            alpha_old = alpha
            f = K.dot(alpha_old)
            w = sigmoid(f) * sigmoid(-f)
            z = f + y / sigmoid(y*(f))
            
            alpha = ksolveRR_2(X, y, lam = 2*self.lam, \
                               sigma=self.sigma, sample_weights = w).fit().alpha
            
            # Break condition (achieved convergence)
            if np.sum((alpha-alpha_old)**2) < self.tol:
                break  
                
                
        self.n_iter = n_iter
        self.alpha = alpha
        
        return self
    
        
    def predict(self, X, threshold):
        K_x = self.kernel(X, self.X)
        return np.where(K_x.dot(self.alpha) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

time: 34.8 ms


In [26]:
# You don't need to look at this, this is just to adapt our matrices
# to the solver being used
solver='cvxopt'

import cvxopt

def cvxopt_qp(P, q, G, h, A, b):
    P = .5 * (P + P.T)
    cvx_matrices = [
        cvxopt.matrix(M) if M is not None else None for M in [P, q, G, h, A, b] 
    ]
    cvxopt.solvers.options['show_progress'] = False
    solution = cvxopt.solvers.qp(*cvx_matrices, options={'show_progress': False})
    return np.array(solution['x']).flatten()

solve_qp = cvxopt_qp

# def quadprog_solve_qp(P, q, G=None, h=None, A=None, b=None):
#     qp_G = .5 * (P + P.T)   # make sure P is symmetric
#     qp_a = -q
#     if A is not None:
#         qp_C = -np.vstack([A, G]).T
#         qp_b = -np.hstack([b, h])
#         meq = A.shape[0]
#     else:  # no equality constraint
#         qp_C = - G.T
#         qp_b = - h
#         meq = 0
#     return quadprog.solve_qp(qp_G, qp_a, qp_C, qp_b, meq)[0]



# solve_qp = {'quadprog': quadprog_solve_qp, 'cvxopt': cvxopt_qp}[solver]

def svm_dual_soft_to_qp_kernel(K, y, C=1):
    n = K.shape[0]
    assert (len(y) == n)
        
    # Dual formulation, soft margin
    P = np.diag(y).dot(K).dot(np.diag(y))
    # As a regularization, we add epsilon * identity to P
    eps = 1e-12
    P += eps * np.eye(n)
    q = - np.ones(n)
    G = np.vstack([-np.eye(n), np.eye(n)])
    h = np.hstack([np.zeros(n), C * np.ones(n)])
    A = y[np.newaxis, :]
    A = A.astype('float')
    b = np.array([0.])
    return P, q, G, h, A, b

# SVM primal soft
class KernelSVM():
    def __init__(self, X, y, C=0.1, lam = 0.1, tol = 1e-3, kernel=None):
        self.alpha = None
        self.X = X
        self.y = y
        self.w = None
        self.b = None
        self.C = C
        self.kernel = kernel
        self.lam = lam        
        self.tol = tol
    
    def fit(self):
        
        X = self.X
        y = self.y
        C = self.C
        
        n, p = X.shape
        assert (len(y) == n)
        K = self.kernel(X, X)
        
        # Solve dual problem
        self.alpha = solve_qp(*svm_dual_soft_to_qp_kernel(K, y, C=self.C))
        
        
       # Compute support vectors and bias b
        sv = np.logical_and((self.alpha>self.tol), \
                            (self.C - self.alpha > self.tol))
        
        self.bias = y[sv] - K[sv].dot(self.alpha*y)
        self.bias =  self.bias.mean()

        self.support_vector_indices = np.nonzero(sv)[0]
        
        return self
        
        
        
    def predict(self, X, threshold):
        K_x = self.kernel(X, self.X)
        return np.where((K_x.dot(self.alpha * self.y) +  self.bias) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

time: 87.9 ms


# Cross Validation

In [27]:
X_train_ = pd.read_csv('./data/Xtr.csv', sep=',') #we use this dataset to train our model
Y_train_ = pd.read_csv('./data/Ytr.csv', sep=',') #we use this dataset to train our model
X_test_ = pd.read_csv('./data/Xte.csv', sep=',')

kfold=KFold(n_splits=5)

def spectrum_kernal(X_train, y, X_test, n=2, encoder=8, one_hot = True, normalise = False):
    
    d = {'A': 1, 'C':2, 'G':3, 'T':4}
    
    for i in range(0, 101-n+1, 1):
        X_train['seq_'+str(i)] = X_train.seq.apply(lambda x :x[i:i+n])
        X_test['seq_'+str(i)] = X_test.seq.apply(lambda x :x[i:i+n])
        
        X_train['seq_'+str(i)] = X_train['seq_'+str(i)].apply(lambda x : sum([d[x[ii]]*encoder**(ii+1) for ii in range(n)]))
        X_test['seq_'+str(i)] = X_test['seq_'+str(i)].apply(lambda x : sum([d[x[ii]]*encoder**(ii+1) for ii in range(n)]))
        
        
        
    X = X_train.drop(['seq', 'Id'], axis=1)
    X_t = X_test.drop(['seq', 'Id'], axis=1)
    y = Y_train.Bound
    
#     print(f'Train: \n{X.tail()}\n -----------------------\n')
#     print(f'Test: \n {X_t.tail()}')

    if one_hot:
        onehot_encoder = OneHotEncoder(sparse=False, categories='auto', handle_unknown='ignore')

        X_cross = X.values
        X_t = X_t.values
        
        enc = onehot_encoder.fit(X)
        X_cross = enc.transform(X)
        X_t_enc = enc.transform(X_t)
        
    elif normalise:
        scaler = MinMaxScaler() #MinMaxScaler() # StandardScaler()
        scaler.fit(X)
        
        X_cross = scaler.transform(X)
        X_t_enc = scaler.transform(X_t)
        
    else :
        
        X_cross = X.values
        X_t_enc = X_t.values
    
    y_cross = y.values
    
    return X_cross, y_cross, X_t_enc

time: 62.7 ms


# Hyper Parameters search 

In [28]:
def objective_sgd(trial):
    
    q  = trial.suggest_loguniform('q', 1e-5, 3e+0)
    
    sigma  = trial.suggest_loguniform('sigma', 1e-1, 8e+0) # trial.suggest_float('sigma', 1e-5, 1e-3, log=True)
    
    kernel = trial.suggest_categorical('kernel', [0, 8, 4])#[Exponential, Laplacian, RationalQuadratic, InverseMultiquadratic, \
                                               #Cauchy, TStudent, ANOVA, Fourier, Tanimoto, Sorensen])
    
#     lam = trial.suggest_loguniform('lam', 1e-15, 1e-4)
    
#     d = trial.suggest_int('d', 2, 8)
    c = trial.suggest_loguniform('c', 1e-2, 5)
    
    degree = trial.suggest_int('degree', 2, 10)
    
    # trick to avoid warnning from opma that support only basic datatype
    trick_list = [rbf_kernel, Exponential, Laplacian, RationalQuadratic, InverseMultiquadratic, \
                        Cauchy, TStudent, ANOVA, Fourier, Tanimoto, Sorensen]
        
    n = trial.suggest_int('n', 1, 3)
    
    tol = 1e-3 # trial.suggest_loguniform('tol', 1e-7, 1e-0)

     
    kernels = {rbf_kernel : 'rbf',
               Exponential : 'Exponential kernel (self, sigma=None)',
               Laplacian : 'Laplacian kernel (self, sigma=None)',
               Cauchy : 'Cauchy kernel (self, sigma=None)',
               
               RationalQuadratic : 'Rational quadratic kernel (self, c=1)',
               InverseMultiquadratic : 'Inverse multiquadratic kernel (self, c=1)',
               
               TStudent : 'T-Student kernel (self, degree=2)',
               
               ANOVA : 'ANOVA kernel (self, sigma=1., d=2)',
               
               Fourier : 'Fourier kernel (self, q=0.1)',
               
               Tanimoto : 'Tanimoto kernel',
               Sorensen : 'Sorensen kernel'
              }
    
    if kernels[trick_list[kernel]] == 'ANOVA kernel (self, sigma=sigma, d=d)':
        kernel = trick_list[kernel](sigma=sigma, d=d)
        
    elif kernels[trick_list[kernel]] == 'Exponential kernel (self, sigma=None)' or \
                        kernels[trick_list[kernel]] == 'Laplacian kernel (self, sigma=None)' or \
                        kernels[trick_list[kernel]] == 'Cauchy kernel (self, sigma=None)'or \
                        kernels[trick_list[kernel]] == 'rbf':
        
        kernel = trick_list[kernel](sigma=sigma)
        
    elif kernels[trick_list[kernel]] == 'Rational quadratic kernel (self, c=1)' or \
                        kernels[trick_list[kernel]] == 'Inverse multiquadratic kernel (self, c=1)':
        kernel = trick_list[kernel](c=c)
        
    elif kernels[trick_list[kernel]] == 'T-Student kernel (self, degree=2)':
        kernel = trick_list[kernel](degree=degree)
        
    elif kernels[trick_list[kernel]] == 'Fourier kernel (self, q=0.1)':
        kernel = trick_list[kernel](q=q)
        
    else:
        kernel = trick_list[kernel]()
        
    
#     n = 1
    
    models = {KernelSVM : 'Kernal SVM'}
    

    X_cross, y_cross, X_t_enc = spectrum_kernal(X_train_, Y_train_, X_test_, n=n, encoder=8, one_hot = True, normalise = False)
    
    for model in models:
        accuracy = []
        for i, (train_index, validate_index) in enumerate(kfold.split(X_cross)):
            X_train, y_train = X_cross[train_index], y_cross[train_index]
            X_valid, y_valid = X_cross[validate_index], y_cross[validate_index]

            if models[model] == 'weigh Ridge Reg':
                sample_weights = np.random.rand(len(y_train))
                model_curr = model(X_train, y_train, lam = lam, sigma = sigma, sample_weights = sample_weights, kernel = kernel)

            elif models[model] == 'k Logistic Ridge Reg':
                model_curr = model(X_train, y_train, lam = lam, sigma = sigma, max_iter=100, tol = tol, kernel = kernel)
            elif models[model] == 'k Ridge Reg':
                
                model_curr = model(X_train, y_train, lam= lam, kernel=kernel)
            elif models[model] == 'log reg':
                model_curr = model(lr=lr, num_iter=5000, batch_size=1)
            else:
                model_curr = model(X_train, y_train, C=c, lam = lam, tol = tol, kernel = kernel)
                
            if models[model] == 'log reg':
                model_curr.fit(X_train, y_train)
            else:
                model_curr.fit()
            

            accuracy.append(model_curr.Accuracy_check(X_valid, y_valid, threshold=0.5))

    return np.mean(accuracy)

# Uncomment the code bellow to run hypperparameters search 

# sampler = optuna.samplers.TPESampler()

# study = optuna.create_study(sampler=sampler, direction='maximize')
# study.optimize(func=objective_sgd, n_trials=1500, show_progress_bar=True)


# trial = study.best_trial

# print('Accuracy: {}'.format(trial.value))
# print("Best hyperparameters: {}".format(trial.params))

time: 106 ms


In [29]:
# print('Accuracy: {}'.format(trial.value))
# print("Best hyperparameters: {}".format(trial.params))

time: 592 µs


In [30]:
params = {'sigma': 4.119788517147901, 'lam': 1.2752298700618223e-14} 

time: 950 µs


In [31]:
# c  = 0.1

# trial.params
# sigma  = 5
# kenel = 0
# lam = 0.0001

sigma  = params['sigma'] #4.133127895830191 #4#4.119788517147901 #4.538118805230398
kenel = 0 #params['kenel']
# q = params['q']
# d = params['lam']
# c = params['c']
# degree = params['degree']
lam = params['lam']
# model = params['models']

trick_list = [rbf_kernel, Exponential, Laplacian, RationalQuadratic, InverseMultiquadratic, \
                        Cauchy, TStudent, ANOVA, Fourier, Tanimoto, Sorensen]
      
kernels = {rbf_kernel : 'rbf',
           Exponential : 'Exponential kernel (self, sigma=None)',
           Laplacian : 'Laplacian kernel (self, sigma=None)',
           Cauchy : 'Cauchy kernel (self, sigma=None)',

           RationalQuadratic : 'Rational quadratic kernel (self, c=1)',
           InverseMultiquadratic : 'Inverse multiquadratic kernel (self, c=1)',

           TStudent : 'T-Student kernel (self, degree=2)',

           ANOVA : 'ANOVA kernel (self, sigma=1., d=2)',

           Fourier : 'Fourier kernel (self, q=0.1)',

           Tanimoto : 'Tanimoto kernel',
           Sorensen : 'Sorensen kernel'
          }

if kernels[trick_list[kenel]] == 'ANOVA kernel (self, sigma=sigma, d=d)':
        kenel = trick_list[kenel](sigma=sigma, d=d)
        
elif kernels[trick_list[kenel]] == 'Exponential kernel (self, sigma=None)' or \
                    kernels[trick_list[kenel]] == 'Laplacian kernel (self, sigma=None)' or \
                    kernels[trick_list[kenel]] == 'Cauchy kernel (self, sigma=None)'or \
                    kernels[trick_list[kenel]] == 'rbf':

    kenel = trick_list[kenel](sigma=sigma)

elif kernels[trick_list[kenel]] == 'Rational quadratic kernel (self, c=1)' or \
                    kernels[trick_list[kenel]] == 'Inverse multiquadratic kernel (self, c=1)':
    kenel = trick_list[kenel](c=c)

elif kernels[trick_list[kenel]] == 'T-Student kernel (self, degree=2)':
    kenel = trick_list[kenel](degree=degree)

elif kernels[trick_list[kenel]] == 'Fourier kernel (self, q=0.1)':
    kenel = trick_list[kenel](q=q)

else:
    kenel = trick_list[kenel]()


n = 1

models = {ksolveRR : 'k Ridge Reg'} 

X_cross, y_cross, X_t_enc = spectrum_kernal(X_train_, Y_train_, X_test_, n=n, encoder=8, one_hot = True, normalise = False)

for model in models:
    accuracy = []
    for i, (train_index, validate_index) in enumerate(kfold.split(X_cross)):
        X_train, y_train = X_cross[train_index], y_cross[train_index]
        X_valid, y_valid = X_cross[validate_index], y_cross[validate_index]

        if models[model] == 'weigh Ridge Reg':
            sample_weights = np.random.rand(len(y_train))
            model_curr = model(X_train, y_train, lam = lam, sigma = sigma, sample_weights = sample_weights, kernel = kenel)

        elif models[model] == 'k Logistic Ridge Reg':
            model_curr = model(X_train, y_train, lam = lam, sigma = sigma, max_iter=100, tol = tol, kernel = kenel)
        elif models[model] == 'k Ridge Reg':
            model_curr = model(X_train, y_train, lam= lam, kernel=kenel)
        elif models[model] == 'log reg':
            model_curr = model(lr=lr, num_iter=5000, batch_size=1)
        else:
            model_curr = model(X_train, y_train, C=c, lam = lam,\
                               tol= tol, kernel = kenel)

        if models[model] == 'log reg':
            model_curr.fit(X_train, y_train)
        else:
            model_curr.fit()


        accuracy.append(model_curr.Accuracy_check(X_valid, y_valid, threshold=0.5))
        print(f'accurracy fold {i}: {accuracy[i]}')

print(f'\nAverage accuracy {models[model]} is : {np.mean(accuracy)}\n')

accurracy fold 0: 0.5975
accurracy fold 1: 0.6625
accurracy fold 2: 0.68
accurracy fold 3: 0.6475
accurracy fold 4: 0.7

Average accuracy k Ridge Reg is : 0.6575

time: 2.19 s


In [34]:
# Training the model with full data

sigma  = params['sigma'] #4.133127895830191 #4#4.119788517147901 #4.538118805230398
kenel = 0 #params['kenel']
# q = params['q']
# d = params['lam']
# c = params['c']
# degree = params['degree']
lam = params['lam']

X_cross, y_cross, X_t_enc = spectrum_kernal(X_train_, Y_train_, X_test_, n=n, encoder=8, one_hot = True, normalise = False)


trick_list = [rbf_kernel, Exponential, Laplacian, RationalQuadratic, InverseMultiquadratic, \
                        Cauchy, TStudent, ANOVA, Fourier, Tanimoto, Sorensen]
       
kernels = {rbf_kernel : 'rbf',
           Exponential : 'Exponential kernel (self, sigma=None)',
           Laplacian : 'Laplacian kernel (self, sigma=None)',
           Cauchy : 'Cauchy kernel (self, sigma=None)',

           RationalQuadratic : 'Rational quadratic kernel (self, c=1)',
           InverseMultiquadratic : 'Inverse multiquadratic kernel (self, c=1)',

           TStudent : 'T-Student kernel (self, degree=2)',

           ANOVA : 'ANOVA kernel (self, sigma=1., d=2)',

           Fourier : 'Fourier kernel (self, q=0.1)',

           Tanimoto : 'Tanimoto kernel',
           Sorensen : 'Sorensen kernel'
          }

if kernels[trick_list[kenel]] == 'ANOVA kernel (self, sigma=sigma, d=d)':
        kenel = trick_list[kenel](sigma=sigma, d=d)
        
elif kernels[trick_list[kenel]] == 'Exponential kernel (self, sigma=None)' or \
                    kernels[trick_list[kenel]] == 'Laplacian kernel (self, sigma=None)' or \
                    kernels[trick_list[kenel]] == 'Cauchy kernel (self, sigma=None)'or \
                    kernels[trick_list[kenel]] == 'rbf':

    kenel = trick_list[kenel](sigma=sigma)

elif kernels[trick_list[kenel]] == 'Rational quadratic kernel (self, c=1)' or \
                    kernels[trick_list[kenel]] == 'Inverse multiquadratic kernel (self, c=1)':
    kenel = trick_list[kenel](c=c)

elif kernels[trick_list[kenel]] == 'T-Student kernel (self, degree=2)':
    kenel = trick_list[kenel](degree=degree)

elif kernels[trick_list[kenel]] == 'Fourier kernel (self, q=0.1)':
    kenel = trick_list[kenel](q=q)

else:
    kenel = trick_list[kenel]()


n = 1

models = {ksolveRR : 'k Ridge Reg'} 


for model in models:
    if models[model] == 'weigh Ridge Reg':
        sample_weights = np.random.rand(len(y_train))
        model_curr = model(X_cross, y_cross, lam = lam, sigma = sigma, sample_weights = sample_weights, kernel = kenel)

    elif models[model] == 'k Logistic Ridge Reg':
        model_curr = model(X_cross, y_cross, lam = lam, sigma = sigma, max_iter=100, tol = tol, kernel = kenel)
    
    elif models[model] == 'k Ridge Reg':
        model_curr = model(X_cross, y_cross, lam= lam, kernel=kenel)
    
    elif models[model] == 'log reg':
        model_curr = model(lr=lr, num_iter=5000, batch_size=1)
    
    else:
        model_curr = model(X_cross, y_cross, C=c, lam = lam,\
                           tol= tol, kernel = kenel)

    if models[model] == 'log reg':
        model_curr.fit(X_cross, y_cross)
    
    else:
        model_curr.fit()
            

model_curr.Accuracy_check(X_cross, y_cross, threshold=0.5)

1.0

time: 1.42 s


# Predictions

In [18]:
# Training the model with full data

X_cross, y_cross, X_t_enc = spectrum_kernal(X_train_, Y_train_, X_test_, n=n, encoder=8, one_hot = True, normalise = False)


trick_list = [rbf_kernel, Exponential, Laplacian, RationalQuadratic, InverseMultiquadratic, \
                        Cauchy, TStudent, ANOVA, Fourier, Tanimoto, Sorensen]
       
kernels = {rbf_kernel : 'rbf',
           Exponential : 'Exponential kernel (self, sigma=None)',
           Laplacian : 'Laplacian kernel (self, sigma=None)',
           Cauchy : 'Cauchy kernel (self, sigma=None)',

           RationalQuadratic : 'Rational quadratic kernel (self, c=1)',
           InverseMultiquadratic : 'Inverse multiquadratic kernel (self, c=1)',

           TStudent : 'T-Student kernel (self, degree=2)',

           ANOVA : 'ANOVA kernel (self, sigma=1., d=2)',

           Fourier : 'Fourier kernel (self, q=0.1)',

           Tanimoto : 'Tanimoto kernel',
           Sorensen : 'Sorensen kernel'
          }

if kernels[trick_list[kenel]] == 'ANOVA kernel (self, sigma=sigma, d=d)':
        kenel = trick_list[kenel](sigma=sigma, d=d)
        
elif kernels[trick_list[kenel]] == 'Exponential kernel (self, sigma=None)' or \
                    kernels[trick_list[kenel]] == 'Laplacian kernel (self, sigma=None)' or \
                    kernels[trick_list[kenel]] == 'Cauchy kernel (self, sigma=None)'or \
                    kernels[trick_list[kenel]] == 'rbf':

    kenel = trick_list[kenel](sigma=sigma)

elif kernels[trick_list[kenel]] == 'Rational quadratic kernel (self, c=1)' or \
                    kernels[trick_list[kenel]] == 'Inverse multiquadratic kernel (self, c=1)':
    kenel = trick_list[kenel](c=c)

elif kernels[trick_list[kenel]] == 'T-Student kernel (self, degree=2)':
    kenel = trick_list[kenel](degree=degree)

elif kernels[trick_list[kenel]] == 'Fourier kernel (self, q=0.1)':
    kenel = trick_list[kenel](q=q)

else:
    kenel = trick_list[kenel]()


n = 1

models = {ksolveRR : 'k Ridge Reg'} 


for model in models:
    if models[model] == 'weigh Ridge Reg':
        sample_weights = np.random.rand(len(y_train))
        model_curr = model(X_cross, y_cross, lam = lam, sigma = sigma, sample_weights = sample_weights, kernel = kenel)

    elif models[model] == 'k Logistic Ridge Reg':
        model_curr = model(X_cross, y_cross, lam = lam, sigma = sigma, max_iter=100, tol = tol, kernel = kenel)
    
    elif models[model] == 'k Ridge Reg':
        model_curr = model(X_cross, y_cross, lam= lam, kernel=kenel)
    
    elif models[model] == 'log reg':
        model_curr = model(lr=lr, num_iter=5000, batch_size=1)
    
    else:
        model_curr = model(X_cross, y_cross, C=c, lam = lam,\
                           tol= tol, kernel = kenel)

    if models[model] == 'log reg':
        model_curr.fit(X_cross, y_cross)
    
    else:
        model_curr.fit()
            
# Prediction 
y_pred = model_curr.predict(X_t_enc, 0.5)

time: 1.36 s


In [19]:
X = np.arange(1000).reshape(-1, 1)
sample = pd.DataFrame(data=X, columns=['Id'])
sample.head()

,Id
0,0
1,1
2,2
3,3
4,4


time: 12.6 ms


In [20]:
sample['Bound'] = y_pred

time: 2.1 ms


In [21]:
sample.tail()

,Id,Bound
995,995,0
996,996,0
997,997,1
998,998,1
999,999,1


time: 14.7 ms


In [22]:
sample.to_csv('./ksolveRR_accuracy'+str(np.mean(accuracy))+'_cv_rbf_kernel_sigma_'+str(sigma)+'_lam_'+str(lam)+'.csv', index=False)

time: 76.5 ms
